In [1]:
import pandas as pd
import numpy as np
import tqdm
import glob
import os
import joblib
import os
import re
import os
import tqdm
from glob import glob
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import graphviz

from collections import defaultdict
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline

class QuestionModel:
    def __init__(self, html_files_path, scores_file_path, prompt_question_label_file_path):
        self.html_files_path = html_files_path
        self.scores_file_path = scores_file_path
        self.prompt_question_label_file_path = prompt_question_label_file_path
    
        self.models = {}
        self.evaluation = {}
        self.clf = None
        self.vectorizer = None



        code2convos = dict()
        pbar = tqdm.tqdm(sorted(list(glob(self.html_files_path))))
        for path in pbar:
            # print(Path.cwd() / path)
            file_code = os.path.basename(path).split(".")[0]
            with open(path, "r", encoding="latin1") as fh:
                    
                # get the file id to use it as key later on
                fid = os.path.basename(path).split(".")[0]

                # read the html file
                html_page = fh.read()

                # parse the html file with bs4 so we can extract needed stuff
                soup = BeautifulSoup(html_page, "html.parser")

                # grab the conversations with the data-testid pattern
                data_test_id_pattern = re.compile(r"conversation-turn-[0-9]+")
                conversations = soup.find_all("div", attrs={"data-testid": data_test_id_pattern})

                convo_texts = []

                for i, convo in enumerate(conversations):
                    convo = convo.find_all("div", attrs={"data-message-author-role":re.compile( r"[user|assistant]") })
                    if len(convo) > 0:
                        role = convo[0].get("data-message-author-role")
                        convo_texts.append({
                                "role" : role,
                                "text" : convo[0].text
                            }
                        )
                        
                code2convos[file_code] = convo_texts
        

        prompts = []
        answers = []
        code2prompts = defaultdict(list)
        code2answers = defaultdict(list)
        for code , convos in code2convos.items():
            user_prompts = []
            for conv in convos:
                if conv["role"] == "user":
                    prompts.append(conv["text"].lower())
                    user_prompts.append(conv["text"].lower()) # Adding the lower case version of the prompt
                else:
                    answers.append(conv["text"].lower())
                    code2answers[code].append(conv["text"].lower()) # Adding the lower case version of the answer

            code2prompts[code] = user_prompts
        # mapping prompts to answers
        code2prompt_answer_pairs = defaultdict(list)

        for code in code2convos:
            for prompt, answer in zip(code2prompts[code], code2answers[code]):
                code2prompt_answer_pairs[code].append((prompt, answer))


        # Converting the dictionary to a DataFrame
        refactored_data = []
        for code, pairs in code2prompt_answer_pairs.items():
            vectorized_pairs = [(prompt.split(), answer.split()) for prompt, answer in pairs]
            refactored_data.append({'code': code, 'prompt_answer_pairs': vectorized_pairs})

        df = pd.DataFrame(refactored_data)

        # reading the scores
        scores = pd.read_csv("data/scores.csv", sep=",")
        scores["code"] = scores["code"].apply(lambda x: x.strip())

        # selecting the columns we need and we care
        scores = scores[["code", "grade"]]

        # join the scores with the df
        df = df.merge(scores, on="code")

        # adding a new column named chat_length and assign it to the size of the prompt_answer_pairs
        df["chat_length"] = df["prompt_answer_pairs"].apply(lambda x: len(x))
        #sort by grade
        df = df.sort_values(by=["grade"], ascending=False)

        #init a new df from scratch
        new_df = pd.DataFrame(columns=["prompt", "which_question", "grade"])

        for row in df.itertuples():
            for prompt, answer in row.prompt_answer_pairs:
                #removing stop words
                prompt = remove_stopwords(prompt)
                #convert prompt to string
                promptStr = " ".join(prompt)
                #add it to a new a new df without using append
                qNo = predict_question_number(promptStr,clf,vectorizer)

                new_df.loc[len(new_df.index)] = [promptStr, qNo, row.grade]


        # replace Nan values with the mean of the column for column grade
        new_df["grade"].fillna((new_df["grade"].mean()), inplace=True)











    def train_classifier(self, df):
        """
        Train a classifier to predict the question number based on the prompt.
        """
        X_train, X_test, y_train, y_test = train_test_split(
            df['prompt'].str.lower(),
            df['related_question'],
            test_size=0.2,
            random_state=42
        )

        self.vectorizer = TfidfVectorizer()
        X_train_vect = self.vectorizer.fit_transform(X_train)
        X_test_vect = self.vectorizer.transform(X_test)

        self.clf = MultinomialNB()
        self.clf.fit(X_train_vect, y_train)

        # Calculate and print the accuracy
        y_pred = self.clf.predict(X_test_vect)
        accuracy = accuracy
